In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from ydata_profiling import ProfileReport



df= pd.read_csv(r"D:\TASKS-NALA\master-tasks\Estadistica Aplicada\Parcial01\tarea02\CarPricesPrediction.csv")
df.head(10)


C:\Users\crasc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Make,Model,Year,Mileage,Condition,Price
0,Ford,Silverado,2022.0,18107.0,Excellent,19094.75
1,Toyota,Silverado,2014.0,13578.0,Excellent,27321.10
2,Chevrolet,Civic,2016.0,46054.0,Good,23697.30
3,Ford,Civic,2022.0,34981.0,Excellent,18251.05
4,Chevrolet,Civic,2019.0,63565.0,Excellent,19821.85
5,Ford,Silverado,2013.0,23600.0,Excellent,27820.00
6,Chevrolet,Silverado,2015.0,89471.0,Good,22526.55
7,Ford,Camry,2014.0,121298.0,Good,21935.10
8,Ford,Altima,2016.0,110691.0,Excellent,20465.45
9,Toyota,Altima,2019.0,112693.0,Excellent,17365.45


In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,909.0,2015.844884,3.713757,2010.0,2013.0,2016.0,2019.0,2022.00
Mileage,909.0,78648.289329,37174.329531,12784.0,47483.0,77857.0,109321.0,146622.00
Price,927.0,22219.137540,3981.484014,14197.5,19100.4,22276.8,25392.1,30025.55


In [4]:
df.isnull().sum()


Make          0
Model         0
Year         18
Mileage      18
Condition     0
Price         0
dtype: int64

### Descripción del Dataset

El dataset utilizado en este análisis contiene información sobre los precios de automóviles y sus características asociadas. Aquí hay una breve descripción general del dataset:

**Columnas:** El dataset consta de varias columnas, incluyendo:

*   **Make:** La marca o fabricante del automóvil (ej., Toyota, Honda, Ford).
*   **Model:** El modelo específico del automóvil (ej., Camry, Civic, F-150).
*   **Year:** El año de fabricación del automóvil.
*   **Mileage:** El kilometraje total (en millas) del automóvil.
*   **Condition:** La condición del automóvil, categorizada como Excelente, Buena o Regular.
*   **Price:** El precio del automóvil.

**Tamaño:** El dataset contiene un cierto número de filas, cada una representando una entrada de automóvil única, y un conjunto de columnas que describen varios atributos de los automóviles.

**Fuente:** El dataset fue generado sintéticamente para el propósito de este análisis. Fue creado utilizando un script de Python que simuló los precios de los automóviles basándose en valores aleatorios y factores predefinidos para imitar la variabilidad del mundo real.

**Propósito:** El dataset se utiliza para el análisis exploratorio de datos (EDA) y tareas de modelado. Sirve como un dataset de muestra para demostrar técnicas de análisis de datos, como la limpieza de datos, la visualización y el modelado predictivo, en un contexto de predicción de precios de automóviles.

**Tipos de Datos:** El dataset consta de tipos de datos tanto numéricos como categóricos. Las características numéricas incluyen Year, Mileage y Price, mientras que las características categóricas incluyen Make, Model y Condition.

**Valores Faltantes:** No hay valores faltantes en el dataset, asegurando que el análisis se pueda realizar sin problemas sin la necesidad de imputación o manejo de datos faltantes.

En general, este dataset proporciona una base para analizar y comprender los factores que influyen en los precios de los automóviles, explorar las relaciones entre las características y construir modelos predictivos para estimar los precios de los automóviles basándose en los atributos dados.



In [5]:
# Separar el dataset en variables independientes (X) y dependiente (y)
X = df.drop('Price', axis=1)
y = df['Price']

# Cambiar cada dato faltante por el promedio de la columna

# Imputar valores faltantes en 'Year'
X['Year'].fillna(X['Year'].mean(), inplace=True)

# Imputar valores faltantes en 'Mileage'
X['Mileage'].fillna(X['Mileage'].mean(), inplace=True)


# --- Verificación (Opcional) ---
# Puedes ejecutar esta línea para confirmar que ya no hay valores nulos en tus variables independientes.
print("Conteo de nulos en X después de la imputación:")
print(X.isnull().sum())


Conteo de nulos en X después de la imputación:
Make         0
Model        0
Year         0
Mileage      0
Condition    0
dtype: int64


C:\Users\crasc\AppData\Local\Temp\ipykernel_10228\4180102563.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['Year'].fillna(X['Year'].mean(), inplace=True)
C:\Users\crasc\AppData\Local\Temp\ipykernel_10228\4180102563.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

In [6]:
from sklearn.preprocessing import LabelEncoder

# Creamos una copia de X para no modificar el original
X_label_encoded = X.copy()

# Inicializamos el codificador
le = LabelEncoder()

# Aplicamos el encoding a cada columna categórica
for col in ['Make', 'Model', 'Condition']:
    X_label_encoded[col] = le.fit_transform(X_label_encoded[col])

# --- Verificación (Opcional) ---
print("DataFrame con Label Encoding:")
print(X_label_encoded.head())


DataFrame con Label Encoding:
   Make  Model    Year  Mileage  Condition
0     1      4  2022.0  18107.0          0
1     4      4  2014.0  13578.0          0
2     0      2  2016.0  46054.0          2
3     1      2  2022.0  34981.0          0
4     0      2  2019.0  63565.0          0


In [7]:
# Usamos la función get_dummies de pandas, que es muy directa
X_one_hot_encoded = pd.get_dummies(X, columns=['Make', 'Model', 'Condition'])

# --- Verificación (Opcional) ---
print("\nDataFrame con One-Hot Encoding (variables dummy):")
# Usamos .iloc para ver las primeras filas y algunas de las nuevas columnas
print(X_one_hot_encoded.iloc[:, :7].head())


DataFrame con One-Hot Encoding (variables dummy):
     Year  Mileage  Make_Chevrolet  Make_Ford  Make_Honda  Make_Nissan  \
0  2022.0  18107.0           False       True       False        False   
1  2014.0  13578.0           False      False       False        False   
2  2016.0  46054.0            True      False       False        False   
3  2022.0  34981.0           False       True       False        False   
4  2019.0  63565.0            True      False       False        False   

   Make_Toyota  
0        False  
1         True  
2        False  
3        False  
4        False  


In [8]:
profile = ProfileReport(df, title = "Reporte de Analisis Exploratorio de los Datos CarPricePrediction", )
profile

ModuleNotFoundError: No module named 'ipywidgets'